# 🧠 1.2 – Principios SOLID en Acción (TypeScript)

En este notebook aplicaremos los **principios SOLID** en ejemplos prácticos usando **TypeScript con Deno**.

Veremos cómo cada principio contribuye a un código más limpio y extensible.

---
## 🎯 Objetivos
- Aplicar los cinco principios SOLID en código.
- Comparar ejemplos incorrectos con sus versiones correctas.
- Practicar refactorizaciones paso a paso.

In [ ]:
console.log('✅ Notebook 1.2 – Principios SOLID en Acción listo para usar.');

---
## 1️⃣ S – Single Responsibility Principle (SRP)

Cada clase debe tener **una sola responsabilidad o motivo de cambio**.

### ❌ Ejemplo que viola SRP

In [1]:
class Report {
  constructor(public content: string) {}

  print() {
    console.log(this.content);
  }

  saveToFile() {
    Deno.writeTextFileSync('report.txt', this.content);
  }
}

const report = new Report('Informe mensual');
report.print();
report.saveToFile();

Informe mensual


🔍 Esta clase mezcla **impresión** y **persistencia**, dos responsabilidades distintas.

### ✅ Solución aplicando SRP

In [ ]:
class ReportPrinter {
  print(report: Report) { console.log(report.content); }
}

class ReportSaver {
  save(report: Report) { Deno.writeTextFileSync('report.txt', report.content); }
}

const newReport = new Report('Informe SRP');
new ReportPrinter().print(newReport);
new ReportSaver().save(newReport);



Informe SRP


---
## 2️⃣ O – Open/Closed Principle (OCP)

Las clases deben estar **abiertas para extensión**, pero **cerradas para modificación**.

### ❌ Ejemplo que viola OCP

In [ ]:
class Rectangle {
  constructor(public width: number, public height: number) {}
}

class AreaCalculator {
  totalArea(shapes: any[]): number {
    let area = 0;
    for (const s of shapes) {
      if (s instanceof Rectangle) {
        area += s.width * s.height;
      }
    }
    return area;
  }
}

console.log(new AreaCalculator().totalArea([new Rectangle(2, 3)]));

🔍 Si añadimos un nuevo tipo de figura, debemos **modificar** `AreaCalculator`.

### ✅ Solución aplicando OCP

In [3]:
interface Shape { area(): number }

class Circle implements Shape {
  constructor(public radius: number) {}
  area() { return Math.PI * this.radius ** 2; }
}

class Square implements Shape {
  constructor(public side: number) {}
  area() { return this.side ** 2; }
}

class RectangleEjemplo implements Shape {
  constructor(public sideA: number, public sideB: number){}
  area() { return this.sideA * this.sideB}
}

class AreaCalculatorOCP {
  totalArea(shapes: Shape[]): number {
    return shapes.reduce((sum, s) => sum + s.area(), 0);
  }
}

console.log(new AreaCalculatorOCP().totalArea([new Circle(2), new Square(3), new RectangleEjemplo(2,3)]));

27.566370614359172


---
## 3️⃣ D – Dependency Inversion Principle (DIP)

Los módulos de alto nivel no deben depender de implementaciones concretas, sino de **abstracciones**.

### ❌ Ejemplo incorrecto

In [4]:
class EmailService {
  send(message: string) { console.log(`📧 Enviando email: \${message}`); }
}

class UserServiceBad {
  private email = new EmailService();
  notifyUser() { this.email.send('Cuenta creada.'); }
}

new UserServiceBad().notifyUser();

📧 Enviando email: ${message}


🔍 `UserServiceBad` depende directamente de `EmailService`, lo que dificulta cambiar el canal de notificación.

### ✅ Solución aplicando DIP

In [ ]:
interface Notifier {
  send(message: string): void;
}

class EmailNotifier implements Notifier {
  send(message: string) { console.log(`📧 Enviando email: \${message}`); }
}

class SMSNotifier implements Notifier {
  send(message: string) { console.log(`📱 Enviando SMS: \${message}`); }
}

class NotificadorInventado implements Notifier {
  send(mensaje: string) {

  }
}

class WorkflowNotifier implements Notifier {
    constructor(public email: EmailNotifier, public sms: SMSNotifier) {
    }
    send(mensaje: string) {
      this.email.send(mensaje)
      this.sms.send(mensaje)
    }
}

class KafkaNotifier implements Notifier {
  send(mensaje: string) {}
}

class UserService {
  constructor(private notifier: Notifier) {}
  notifyUser() { this.notifier.send('Tu cuenta ha sido creada.'); }
}

new UserService(new WorkflowNotifier(new EmailNotifier(), new SMSNotifier())).notifyUser();
// new UserService(new SMSNotifier()).notifyUser();

📧 Enviando email: ${message}
📱 Enviando SMS: ${message}


---
## 🧩 Ejercicio – Combinando SOLID

Crea un sistema que registre usuarios aplicando SRP y DIP:
1. Define una interfaz `Logger` con el método `log(message: string)`.
2. Implementa `ConsoleLogger` y `FileLogger`.
3. Crea una clase `UserRegistration` que dependa de `Logger`.

In [ ]:
// Escribe tu solución aquí...
interface Logger {
  log(message: string): void;
}

interface Notifier {
  send(message: string): void;
}

class Email2Notifier implements Notifier {
  send(message: string){
    console.log('enviado por email', message)
  }
}

class ConsoleLogger implements Logger {
  log(message: string) { console.log(`🖥️ Log: ${message}`); }
}


class FileLogger implements Logger {
  log(message: string) { Deno.writeTextFileSync('log.txt', message + '\\n'); }
}



class UserRegistration {
    constructor(public logger:Logger){}

    register(name:string) {
        this.logger.log(name)
    }
}

class UserNotifer {
    constructor(public notifier:Notifier){}

    notifiy(message:string) {
        this.notifier.send(message)
    } 

}


class User {
   
    constructor(public name: string, public registration: UserRegistration, public notifier: UserNotifer) {}

    login(){
      this.registration.register(this.name)
    }


}



class InvoiceRegristration {
    constructor(public logger: Logger){}

    register(code:number) {
        this.logger.log(code.toString())
    } 
}

### ✅ Solución propuesta

In [ ]:
// interface Logger {
//   log(message: string): void;
// }

// class ConsoleLogger implements Logger {
//   log(message: string) { console.log(`🖥️ Log: \${message}`); }
// }

// class FileLogger implements Logger {
//   log(message: string) { Deno.writeTextFileSync('log.txt', message + '\\n'); }
// }

// class UserRegistration {
//   constructor(private logger: Logger) {}
//   register(name: string) {
//     this.logger.log(`Usuario registrado: \${name}`);
//   }
// }

new UserRegistration(new ConsoleLogger()).register('Alice');
new UserRegistration(new FileLogger()).register('Bob');

new InvoiceRegristration(new FileLogger()).register(1000)


let user = new User('David', new UserRegistration(new FileLogger()), new UserNotifer(new Email2Notifier()))

user.login()


🖥️ Log: Alice


---
## 🧠 Resumen del notebook

- **SRP:** una clase → una responsabilidad.
- **OCP:** abiertas a extensión, cerradas a modificación.
- **DIP:** depender de interfaces, no implementaciones.

➡️ Próximo notebook → **2.1 Qué son los Patrones de Diseño**.